优点：数据较少情况下仍然有效，可以处理多类别的问题

缺点：对于输入数据的准备方式较为敏感

使用数据类型：标称型数据


In [16]:
import numpy as np

def loadDataSet():
    postingList=[['my', 'dog','dog' 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]    #1 is abusive, 0 not
    return postingList,classVec

def createVocbList(dataSet):
    '''
    返回字典
    '''
    vocaSet=set()
    for document in dataSet:
        vocaSet = vocaSet | set(document)
    return list(vocaSet)

def bagOfWords2Vec(vocabList, inputSet):
    '''
    word改造成向量
    vocaListabs 字典表
    输入向量
    '''
    ret=[0]*len(vocabList)
    for v in inputSet:
        if v in vocabList:
            ret[vocabList.index(v)]=1
        else:
            print( v, 'not in vocabulary!')
    return ret

In [17]:
postingList,classVec=loadDataSet()
myvocabList=createVocbList(postingList)
print(myvocabList, len(myvocabList))
print(bagOfWords2Vec(myvocabList,postingList[0]))

['garbage', 'help', 'food', 'problems', 'cute', 'ate', 'mr', 'steak', 'buying', 'licks', 'dalmation', 'park', 'doghas', 'dog', 'my', 'how', 'please', 'stop', 'not', 'quit', 'worthless', 'to', 'stupid', 'posting', 'love', 'take', 'so', 'is', 'him', 'I', 'maybe', 'flea'] 32
[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


$$ p(x_i | \vec{w}) = \frac{p(\vec{w}|c_i) p(c_i)}{p(\vec{w})}$$

朴素贝叶斯假设属性独立,则
$$p(\vec{w}|c_i) = p(w_0|c_i)p(w_1|c_i)\dots p(w_N|c_i) $$

上式中，不需要计算P（w）,因为只要分析各个分子项即可

In [21]:
# 11：50-14：20
def trainNB0(trainMatrix,trainCategory):
    '''
    trainmatrix 所有词的矩阵
    trainCategory 标记分类的向量
    '''
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    ## 计算辱骂性出现概率   p(c0)
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    # p0num是不出现辱骂的文档数目  单词数*1的矩阵
    p0Num,p1Num = ones(numWords), ones(numWords) #概率矩阵 
    p0Denom = 0.0; p1Denom = 0.0         #分类

    for i in range(numTrainDocs):
        ##是辱骂性文档
        if trainCategory[i] == 1:
            ## 同时统计各个词汇的内容
            p1Num += trainMatrix[i] ## 各个词汇的出现数量
            p1Denom += sum(trainMatrix[i]) ## 总的词汇数
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    # p(ci)的概率
    p1Vect = log(p1Num/p1Denom)          
    p0Vect = log(p0Num/p0Denom)   
    ## 返回的是 log( p( w| c_i ) )
    return p0Vect,p1Vect,pAbusive

In [24]:
trainMat=[]
for doc in postingList:
    trainMat.append(bagOfWords2Vec(myvocabList,doc))

p0v,p1v,pAb=trainNB0(trainMat,classVec)
print(p0v,p1v,pAb)

[-3.17805383 -2.48490665 -3.17805383 -2.48490665 -2.48490665 -2.48490665
 -2.48490665 -2.48490665 -3.17805383 -2.48490665 -2.48490665 -3.17805383
 -2.48490665 -2.48490665 -1.79175947 -2.48490665 -2.48490665 -2.48490665
 -3.17805383 -3.17805383 -3.17805383 -2.48490665 -3.17805383 -3.17805383
 -2.48490665 -3.17805383 -2.48490665 -2.48490665 -2.07944154 -2.48490665
 -3.17805383 -2.48490665] [-2.2512918  -2.94443898 -2.2512918  -2.94443898 -2.94443898 -2.94443898
 -2.94443898 -2.94443898 -2.2512918  -2.94443898 -2.94443898 -2.2512918
 -2.94443898 -1.84582669 -2.94443898 -2.94443898 -2.94443898 -2.2512918
 -2.2512918  -2.2512918  -1.84582669 -2.2512918  -1.55814462 -2.2512918
 -2.94443898 -2.2512918  -2.94443898 -2.94443898 -2.2512918  -2.94443898
 -2.2512918  -2.94443898] 0.5


In [20]:
## ve2 带分类的向量
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)    #element-wise mult
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else: 
        return 0
    

def testingNB():
    listOPosts,listClasses = loadDataSet()
    myVocabList = createVocbList(listOPosts)
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V,p1V,pAb = trainNB0(array(trainMat),array(listClasses))
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb) )
    testEntry = ['stupid', 'garbage']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb) )
    
    print("===============================")
    for i in range(len(listOPosts)):
        example=array(setOfWords2Vec(myVocabList, listOPosts[i]))
        print('classified as: ',classifyNB(example,p0V,p1V,pAb),\
              listClasses[i] )           
    
testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1
classified as:  0 0
classified as:  1 1
classified as:  0 0
classified as:  1 1
classified as:  0 0
classified as:  1 1


In [9]:
def textParse(bigString):    #input is big string, #output is word list
    import re
    listOfTokens = re.split('[^\w]', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2] 

textParse("sd fsdf sd f ,s fwe,r wer,.w werwer w wer ")

['fsdf', 'fwe', 'wer', 'werwer', 'wer']

In [10]:
emailText=open('email/ham/6.txt').read()

FileNotFoundError: [Errno 2] No such file or directory: 'email/ham/6.txt'

In [11]:
## 开始测试
datas=[]
classVecs=[]

#get spam
for i in range(25):
    datas.append( textParse( open('email/spam/%d.txt' % (i+1) ).read() ))
    classVecs.append(1)
#get ham
for i in range(25):
    datas.append( textParse( open('email/ham/%d.txt' % (i+1) ,  encoding='ISO-8859-1').read() ))
    classVecs.append(0)
    
## 生成字典
vocabs = createVocbList(datas)


vdata=[]
for d in datas:
    vdata.append(bagOfWords2Vec(vocabs, d))

p0v,p1v,v=trainNB0(vdata,classVecs)


FileNotFoundError: [Errno 2] No such file or directory: 'email/spam/1.txt'

In [1]:
#测试数据上的效果
erros=0
for i in range(len(vdata)):
    if classifyNB(vdata[i],p0v,p1v,v) != classVecs[i] :
        erros += 1

print( '%0.2f%% ' % (erros*100.0/len(vdata)) )

NameError: name 'vdata' is not defined